0. Download and install modules from : https://github.com/SpikeInterface

In [10]:
import matplotlib
%autosave 180
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from matplotlib.widgets import Slider, Button, RadioButtons
import matplotlib.gridspec as gridspec

%matplotlib tk

import matplotlib.cm as cm
from matplotlib import gridspec

import numpy as np
import os

import h5py


Autosaving every 180 seconds


In [11]:
Imap = np.load('/home/cat/Imap.npy')
print (Imap.shape)
IOKR = np.load('/home/cat/IOKR_bi.npy')
print (IOKR.shape)

IOKR_uni = np.load('/home/cat/IOKR_uni.npy')

(145, 128, 20, 46)
(145, 128, 20, 46)


In [12]:
Imap_mean = Imap.mean(2)
IOKR_mean = IOKR.mean(2)
IOKR_uni_mean = IOKR_uni.mean(2)


In [16]:
scale = 10
vmin = np.min(Imap_mean)/scale
vmax=np.max(Imap_mean)/scale
    #print (vmin,vmax)
    
imgs = []
imgs2= []
for k in range(46):
    ax=plt.subplot(7,7,k+1)
    temp = Imap_mean[:,:,k]-IOKR_mean[:,:,k]
    
    plt.imshow(temp,vmin=vmin, vmax=vmax)
    plt.xticks([])
    plt.yticks([])
    plt.title(str(round(k*0.1-4.5,1)),pad=0.9)
    
    
    temp = Imap_mean[:,:,k]-IOKR_mean[:,:,k]
    idx = np.where(temp>vmax)
    temp[idx]=vmax
    idx = np.where(temp<vmin)
    temp[idx]=vmin   
    temp = temp/Imap_mean[:,:,k]
    imgs.append(temp)
    
    
    
    temp2 = Imap_mean[:,:,k]-IOKR_uni_mean[:,:,k]
    idx = np.where(temp2>vmax)
    temp2[idx]=vmax
    idx = np.where(temp2<vmin)
    temp2[idx]=vmin
    temp2=temp2/Imap_mean[:,:,k]
    
    imgs2.append(temp2)
    
plt.show()

In [18]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

fig = plt.figure(figsize=(4, 4))

frame = 0
vmax = np.max(imgs)
vmin = np.min(imgs)
ax1 = plt.subplot(1,2,1)
im1 = ax1.imshow(imgs[0], vmin=vmin, vmax=vmax)
ax1.set_title("Uni")

vmax = np.max(imgs2)
vmin = np.min(imgs2)
ax2 = plt.subplot(1,2,2)
im2 = ax2.imshow(imgs2[0], vmin=vmin, vmax=vmax)
ax2.set_title("Bi")

plt.suptitle("slice: "+str(round(frame*0.1-4.5,1)))

#plt.colorbar(shrink=0.5)
plt.title('0')

def update(*args):
    global frame
    plt.suptitle("slice: "+str(round(frame*0.1-4.5,1)))
    im1.set_array(imgs[frame])
    im2.set_array(imgs2[frame])

    ax2.set_title("Bi")
    frame += 1
    #frame %= len(a)

    return im1,im2

ani = animation.FuncAnimation(fig, update, frames=45, interval=100, blit=True)

#ani = animation.ArtistAnimation(fig, img, interval=20, blit=True, repeat_delay=0)

ani.save('/home/cat/OKR_both.avi', writer="ffmpeg")

print(time.clock())
plt.show()


IndexError: list index out of range

In [90]:
geom = np.loadtxt('/home/cat/Downloads/geom(2).txt')
templates = np.load('/home/cat/Downloads/templates.npy')
templates = np.load('/home/cat/Downloads/cluster_result_432/templates.npy')
print (templates.shape)



(8, 61, 934)


In [113]:
from scipy.signal import freqz

data_int16 = np.fromfile('/media/cat/4TBSSD/felix/data_int16.bin','int16',20000*934*1)
#data_int16 = np.fromfile('/media/cat/4TBSSD/felix/data.bin','uint16',20000*934*1)
#data_int16 = np.fromfile('/home/cat/Downloads/mea_20190502_140013.raw.all.redone.2sec.bin','int16')
#data_int16 = np.fromfile('/home/cat/Downloads/mea_20190502_140013.raw.all.2sec.bin','int16')
data2d = np.int16(data_int16).reshape(-1,934)
print (data2d.shape)
    
lowcut=300
highcut=5000
fs = 20000
for k in range(data2d.shape[1]):
    filtered = butter_bandpass_filter(data2d[:,k],lowcut,highcut,fs)  
    data2d[:,k] = filtered

(20000, 934)


In [123]:
templates_old = np.load('/media/cat/4TBSSD/felix/tmp/nn_train/augment/templates/templates.npy')
print (templates_old.shape)

(816, 161, 934)


In [127]:
ptps = templates_old.ptp(1).max(1)
idx = np.where(ptps>16.0)[0]
templates=templates_old[idx]
print ("larger templates; ", templates.shape)
unit = 10


#for k in range(geom.shape[0]):
#print (channel_ids)
idx = np.argsort(channel_ids)
width = 200
offset = 6000-50
yscale = 1

width = templates.shape[1]

for k in np.arange(templates.shape[2]):    
    plt.plot(geom[k,0]+np.arange(width)/(width/5), 
             templates[unit,:,k]/yscale+geom[k,1],c='black')
             #data2d[offset:offset+width,k]/yscale+geom[k,1],c='black')
plt.show()

larger templates;  (14, 161, 934)


In [7]:
from pathlib import Path

path = Path('/media/cat/4TBSSD/felix')

rec_paths = sorted([p for p in path.iterdir() if p.suffix == '.h5'])

recompute = True
recordings = []
for i, rec in enumerate(rec_paths):
    recording = se.Mea1kRecordingExtractor(rec)
    if i == 0:
        channel_order = recording.get_channel_ids()
        recording_sorted = recording
    else:
        recording_sorted = se.SubRecordingExtractor(recording, channel_ids=channel_order)
    recordings.append(recording_sorted)

multirec = se.MultiRecordingTimeExtractor(recordings)

multirec.write_to_binary_dat_format('/home/cat/recording.dat')

Chip version: 20170126


In [ ]:
def get_traces(self, channel_ids=None, start_frame=None, end_frame=None):
    if np.array(channel_ids).size > 1:
        if np.any(np.diff(channel_ids) < 0):
            sorted_idx = np.argsort(channel_ids)
            recordings = self._signals[np.sort(channel_ids), start_frame:end_frame]
            return recordings[sorted_idx].astype('float')
        else:
            return (self._signals[np.array(channel_ids), start_frame:end_frame] * self._lsb).astype('float32')
    else:
        return (self._signals[np.array(channel_ids), start_frame:end_frame] * self._lsb).astype('float32')

